In [ ]:
import geopandas as gpd
import pandas as pd
import json
import requests
import time
from tqdm import tqdm

# load county shapefiles

shapefile_path = '../data/raw/tl_2024_us_county/tl_2024_us_county.shp'
counties = gpd.read_file(shapefile_path)
counties = counties.to_crs(epsg=4326)

# load disaster declarations
df = pd.read_csv("../data/raw/DisasterDeclarationsSummaries.csv")

df["GEOID"] = df["fipsStateCode"].astype(str).str.zfill(2) + df["fipsCountyCode"].astype(str).str.zfill(3)

GEOIDS = "../data/cleaned/county_disaster_counts_by_year.json"

NUM2ABR = "../data/raw/num2abr.txt"


with open(GEOIDS, 'r') as f:
    data = json.load(f)

statedict = {}

with open(NUM2ABR, 'r') as s:
    for line in s:
        parts = line.strip().split(maxsplit=1)
        if len(parts) == 2:
            code, state = parts
            statedict[code] = state

# print(statedict.keys())
# print(df['fipsStateCode'])
df['fipsStateCode'] = df['fipsStateCode'].astype(str).map(statedict)
df = df.dropna()
# print(df['fipsStateCode'])

df['fipsStateCode'] = (df['fipsStateCode'] + '-')

df['URLS'] = (df['fipsStateCode'].astype(str) + df['fipsCountyCode'].astype(str).str.zfill(3))

# print(df['URLS'])

# test_url = "https://www.ncei.noaa.gov/access/monitoring/climate-at-a-glance/county/time-series/TX-059/tavg/1/3/1895-2025/zingchart-config.js"
# r = requests.get(test_url)
# print(r.text)

records = []
base_url = "https://www.ncei.noaa.gov/access/monitoring/climate-at-a-glance/county/time-series/{code}/tavg/1/3/1895-2025.json"

for code in tqdm(df['URLS']):
    temp_url = base_url.replace("{code}", code)
    r = requests.get(temp_url)
    data = r.json()
    for year, entry in data["data"].items():

        value = entry.get("value")
        if value is not None:
            records.append({
                "year": int(year[0:4]),
                "value": float(value)
            })
    time.sleep(0.05)


with open("county_temperature_data.json", "w") as f:
    json.dump(records, f, indent=2)

print('done fr')
    



/var/folders/9_/yf7whgjn76l68s_n1hjflsmc0000gn/T/ipykernel_44894/3028467918.py:15: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../data/raw/DisasterDeclarationsSummaries.csv")
  0%|          | 3/2766 [00:03<1:01:08,  1.33s/it]


KeyboardInterrupt: 